In [1]:
import psycopg2 as pg
import sqlalchemy as db
import pandas as pd
import lyricsgenius as genius
import numpy as np
import pickle
from sqlalchemy.orm import sessionmaker

In [4]:
engine = db.create_engine('postgresql://localhost:5432/lyricsdb')
connection = engine.connect()
metadata = db.MetaData()
artists = db.Table('artist_info', metadata, autoload=True, autoload_with=engine)
songs = db.Table('songs', metadata, autoload=True, autoload_with=engine)
artist_info = db.Table('artist_info', metadata, autoload=True, autoload_with=engine)

In [8]:
missing_artists = ["Simon and Garfunkel",
                   "Joan Jett and the Blackhearts", "Smokey Robinson &amp; the Miracles"]
replacing_names = ["Simon & Garfunkel","Joan Jett & the Blackhearts", 
                   "Smokey Robinson & the Miracles"]

In [9]:
# insert = songs.insert().values(artist_id=1000, song_title="Fake Song", lyrics="Fake Lyrics")

In [10]:
# connection.execute(insert)

In [11]:
for old, new in list(zip(missing_artists, replacing_names)):    
    update = artist_info.update().where(artist_info.c.artist_name==old).values(artist_name=new)
    connection.execute(update)

In [41]:
# connection.execute(update)

In [264]:
with open('../artist_data/artist_dataframe.pkl', 'rb') as f:
    artist_df = pickle.load(f)

In [265]:
with open('../artist_data/song_df.pkl', 'rb') as f:
    song_df = pickle.load(f)

In [266]:
song_df

,artist_id,song_title,lyrics
0,1083,Radio 2,
1,1274,Sound Round,I'm young in my camper van\nThe world feels ol...
2,347,Okie from Muskogee,We don't smoke marijuana in Muskogee\nWe don't...
3,347,Mama Tried,The first thing I remember knowing\nWas a lone...
4,347,One Day at a Time,"I'm only human, I'm just a man\nHelp me to bel..."
...,...,...,...
45995,1418,The Real World,Lead vocal - john miles\nOne more compromise I...
45996,1418,Dancing on a Highwire/Spotlight,(Colin Blunstone - Lead Vocal)\nWe're living i...
45997,1418,To One Paradise,Alan Parsons Project\nTales Of Mystery And Ima...
45998,1418,A dream within a dream - original 1976 version,(instrumental)


In [267]:
len(song_df.artist_id.unique())

187

In [124]:
artist_df.head()

,index,artist_name,genre,artist_picture_ranker_url,artist_picture_genius_url,ranker_ranking
0,0,Tupac,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/115/228...,None,0.0
1,1,The Notorious B.I.G.,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/111/221...,None,1.0
2,2,Eminem,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/47/9378...,None,2.0
3,3,Kendrick Lamar,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/3107/62...,None,3.0
4,4,Dr. Dre,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/45/8916...,None,4.0


In [125]:
pulled_artists = artist_df[artist_df["artist_picture_genius_url"].isnull() == False]

In [126]:
pulled_artists

,index,artist_name,genre,artist_picture_ranker_url,artist_picture_genius_url,ranker_ranking
347,347,Merle Haggard,"Country, Classic Rock",https://imgix.ranker.com/node_img/79/1565940/o...,https://images.genius.com/4635920ae9e995aa782f...,208.0
351,351,Alabama,"Country, Classic Rock",https://imgix.ranker.com/node_img/125/2492847/...,https://images.genius.com/9925a0446f58ca0e111a...,194.0
352,352,Willie Nelson,"Country, Classic Rock",https://imgix.ranker.com/user_node_img/120/239...,https://images.genius.com/b3f5d7b3eaffadcfa7a8...,115.0
365,365,Johnny Cash,"Country, Blues, Classic Rock",https://imgix.ranker.com/user_node_img/66/1300...,https://images.genius.com/3e9e29a781c26d78d3b6...,29.0
400,400,Ricky Nelson,"Country, Classic Rock",https://imgix.ranker.com/node_img/96/1908356/o...,https://images.genius.com/132212814c0c0d2f6048...,105.0
...,...,...,...,...,...,...
1279,1278,Black Sabbath,Classic Rock,https://imgix.ranker.com/user_node_img/30/5941...,https://images.genius.com/c2ad345ff3c4aed279bc...,12.0
1280,1279,Aerosmith,Classic Rock,https://imgix.ranker.com/node_img/21/406180/or...,https://images.genius.com/807ca2e7ed69ef2b6425...,13.0
1281,1280,Tom Petty and the Heartbreakers,Classic Rock,https://imgix.ranker.com/node_img/113/2258790/...,https://images.genius.com/db9e023f54ddb0626624...,14.0
1282,1281,Fleetwood Mac,Classic Rock,https://imgix.ranker.com/node_img/50/999083/or...,https://images.genius.com/c96d9275641318df4605...,15.0


In [308]:
query = 'SELECT * FROM songs;'

In [309]:
song_df_db = pd.read_sql(query, connection)

In [342]:
song_df_db

,artist_id,song_title,lyrics
0,1083,Radio 2,
1,1274,Sound Round,I'm young in my camper van\nThe world feels ol...
2,347,Okie from Muskogee,We don't smoke marijuana in Muskogee\nWe don't...
3,347,Mama Tried,The first thing I remember knowing\nWas a lone...
4,347,One Day at a Time,"I'm only human, I'm just a man\nHelp me to bel..."
...,...,...,...
56004,1485,Monkeyboy,All the world is watching you\nAll your friend...
56005,1485,All the Things We Do,Push another little thing into the sky\nPoke a...
56006,1485,Don’t Call Me Brother,All the years you fought so hard\nAll the drea...
56007,1485,With the Morons,Let me tell you what it's all about\nDon't wan...


In [339]:
query = 'SELECT * FROM artist_info;'

In [340]:
artist_df_db = pd.read_sql(query, connection)

In [341]:
artist_df_db

,artist_id,artist_name,genre,artist_picture_ranker_url,artist_picture_genius_url,ranker_ranking
0,0,Tupac,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/115/228...,None,0
1,1,The Notorious B.I.G.,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/111/221...,None,1
2,2,Eminem,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/47/9378...,None,2
3,3,Kendrick Lamar,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/3107/62...,None,3
4,4,Dr. Dre,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/45/8916...,None,4
...,...,...,...,...,...,...
1481,1473,Skid Row,Classic Rock,https://imgix.ranker.com/node_img/149/2965875/...,https://images.genius.com/95e58326fd080b975edf...,249
1482,1475,Krokus,Classic Rock,https://imgix.ranker.com/node_img/70/1394534/o...,https://images.genius.com/ae8a91bd99193f8f98f1...,251
1483,1477,Tommy Bolin,Classic Rock,https://imgix.ranker.com/user_node_img/113/225...,https://images.genius.com/f4d935113307893801b2...,253
1484,1481,Gentle Giant,Classic Rock,https://imgix.ranker.com/node_img/53/1053647/o...,https://images.genius.com/3fdfca0d1a18ded1ecae...,258


In [326]:
artist_df_db = artist_df_db.rename(columns={"index":"artist_id"})

In [330]:
artist_df_db

,artist_id,artist_name,genre,artist_picture_ranker_url,artist_picture_genius_url,ranker_ranking
0,0,Tupac,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/115/228...,None,0.0
1,1,The Notorious B.I.G.,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/111/221...,None,1.0
2,2,Eminem,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/47/9378...,None,2.0
3,3,Kendrick Lamar,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/3107/62...,None,3.0
4,4,Dr. Dre,Rap/Hip-Hop,https://imgix.ranker.com/user_node_img/45/8916...,None,4.0
...,...,...,...,...,...,...
1481,1473,Skid Row,Classic Rock,https://imgix.ranker.com/node_img/149/2965875/...,https://images.genius.com/95e58326fd080b975edf...,249.0
1482,1475,Krokus,Classic Rock,https://imgix.ranker.com/node_img/70/1394534/o...,https://images.genius.com/ae8a91bd99193f8f98f1...,251.0
1483,1477,Tommy Bolin,Classic Rock,https://imgix.ranker.com/user_node_img/113/225...,https://images.genius.com/f4d935113307893801b2...,253.0
1484,1481,Gentle Giant,Classic Rock,https://imgix.ranker.com/node_img/53/1053647/o...,https://images.genius.com/3fdfca0d1a18ded1ecae...,258.0


In [337]:
artist_df_db.to_sql('artist_info', con=connection, if_exists='append', index=False)

In [332]:
def copy_from_file(conn, df, table):
    """
    Here we are going save the dataframe on disk as 
    a csv file, load the csv file  
    and use copy_from() to copy it to the table
    """
    # Save the dataframe to disk
    tmp_df = "./tmp_dataframe.csv"
    df.to_csv(tmp_df, index_label='id', header=False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        os.remove(tmp_df)
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_file() done")
    cursor.close()
    os.remove(tmp_df)

In [335]:
copy_from_file(connection, artist_df_db, "artist_info")

AttributeError: 'Connection' object has no attribute 'cursor'

In [314]:
pulled_artists = artist_df_db[artist_df_db["artist_picture_genius_url"].isnull() == False]

In [315]:
len(pulled_artists.artist_name.unique())

259

In [199]:
24913/88

283.10227272727275

In [137]:
rock_artists = artist_df[artist_df["artist_name"] == "The Clash" ]

In [138]:
rock_artists

,index,artist_name,genre,artist_picture_ranker_url,artist_picture_genius_url,ranker_ranking
864,863,The Clash,"Alternative, Classic Rock",https://imgix.ranker.com/user_node_img/110/218...,https://images.genius.com/8d5abf77cf5eb22e0c8b...,74.0


In [179]:
geniusCreds = "uWxFeeBbZ1fyBYt5T6OyFJzHq0kZ8DG3bR1HooBJY4oqrScFC8cyf42xKfuA4l0d"

In [180]:
api = genius.Genius(geniusCreds, verbose = False, remove_section_headers = True, skip_non_songs = True, excluded_terms = ["(Remix)", "(Live)"])

In [189]:
genius_artist = api.search_artist("santan", max_songs=1);

In [190]:
genius_artist.songs

[('Thiago Silva', 'Dave')]

NameError: name 'lyricsgenius' is not defined